<a href="https://colab.research.google.com/github/pbthang/cs4225-assignment/blob/main/assign2/cs4225_a2_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Setup Spark
# ===============
# Installing Spark needs to be done once each time you re-open this notebook. It should take around 10-30 seconds.
# ===============
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

# unzip the spark file to the current folder
!tar xf spark-3.3.1-bin-hadoop3.tgz

# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

# install findspark using pip
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
# After downloading dataset, you should have a file "network.csv" in your Files (click the folder icon in the left sidebar)
!wget https://bhooi.github.io/teaching/cs4225/bomb/part2/network.csv

--2023-12-11 07:23:16--  https://bhooi.github.io/teaching/cs4225/bomb/part2/network.csv
Resolving bhooi.github.io (bhooi.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to bhooi.github.io (bhooi.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-12-11 07:23:16 ERROR 404: Not Found.



In [ ]:
df = spark.read.option('header', True).csv("network.csv").repartition(1).cache()
df.toPandas()

,src,dst,day
0,172.016.117.052,207.025.071.144,0
1,194.027.251.021,192.168.001.001,0
2,230.001.010.020,172.016.112.050,0
3,010.020.030.040,172.016.112.050,0
4,172.016.114.169,204.034.204.071,0
...,...,...,...
22767,197.218.177.069,172.016.114.169,29
22768,194.027.251.021,192.168.001.001,29
22769,194.027.251.021,192.168.001.001,29
22770,192.168.001.001,194.027.251.021,29


In [ ]:
# Write your code for processing the data here.
from pyspark.sql.functions import countDistinct

df.createOrReplaceTempView("Network")
non_bomb = df.filter(~df.day.isin(["0", "7", "14", "21", "28"])).createOrReplaceTempView("NotBomb")
bomb = spark.sql("""
  SELECT * FROM Network
  WHERE Network.dst NOT IN (
    SELECT dst FROM NotBomb
  )
""")
bomb = bomb.groupBy("dst").agg(countDistinct("src")).withColumnRenamed("count(src)", "srcCount")
bomb = bomb.where(bomb.srcCount >= 13)

bomb.toPandas()